In [1]:
!pip install opencv-python

     ---------------------------------------- 38.8/38.8 MB 1.3 MB/s eta 0:00:00


In [4]:
!pip install mediapipe

  Using cached mediapipe-0.10.14-cp310-cp310-win_amd64.whl (50.8 MB)
     ---------------------------------------- 45.5/45.5 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 200.1/200.1 kB 1.0 MB/s eta 0:00:00
  Using cached jaxlib-0.4.28-cp310-cp310-win_amd64.whl (51.5 MB)
  Using cached jax-0.4.28-py3-none-any.whl (1.9 MB)


In [1]:
# importing the libraries 
import mediapipe as mp
#open cv
import cv2
import numpy as np
from mediapipe.framework.formats import landmark_pb2 #return the coordinates of the fingers
import time
import random

In [2]:
mp_drawing = mp.solutions.drawing_utils

In [3]:
mp_hands = mp.solutions.hands

In [4]:
score = 0

In [5]:
x_enemy = random.randint(50,600)
y_enemy = random.randint(50,400)

In [7]:
def enemy():
    global score, x_enemy, y_enemy
    cv2.circle(image, (x_enemy, y_enemy), 25, (0,200,0),5)
    
video = cv2.VideoCapture(0)

#More the senstivity more wrong detection
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while video.isOpened():
        _,frame = video.read()
        
        #converting the input video BGR to RGB (opencv works on BGR) (mediapipe works on RGB)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image,1)
        
        #mediapipe return the coordinates in ratio and opencv wants x,y coordinates
        imageHeight, imageWidth, _ = image.shape
        results = hands.process(image)
        
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        
        font= cv2.FONT_HERSHEY_SIMPLEX
        color=(255,0,255)
        text = cv2.putText(image, "Score", (480,30), font, 1, color, 4, cv2.LINE_AA)
        text = cv2.putText(image,str(score), (590,30), font, 1, color,4,cv2.LINE_AA)
        
        enemy()
        
        
        
        if results.multi_hand_landmarks:
#             hand variable has ratio of hand
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(250,44,250),thickness=2, circle_radius=2),)
                
        if results.multi_hand_landmarks != None:
            for handLandmarks in results.multi_hand_landmarks:
                for point in mp_hands.HandLandmark:
                    
                    normalizedLandmark = handLandmarks.landmark[point]
                    pixelCoordinatesLandmark = mp_drawing._normalized_to_pixel_coordinates(normalizedLandmark.x, normalizedLandmark.y, imageWidth, imageHeight)
                    
                    point = str(point)
                    if point == 'HandLandmark.INDEX_FINGER_TIP':
                        try:
                            cv2.circle(image, (pixelCoordinatesLandmark[0], pixelCoordinatesLandmark[1]), 25, (0,200,0), 5) #25 is the radius of circle, () is color and 5 is pen size
                            if pixelCoordinatesLandmark[0]==x_enemy or pixelCoordinatesLandmark[0]==x_enemy+10 or pixelCoordinatesLandmark[0]==x_enemy-10:
                                print("Found")
                                x_enemy = random.randint(50,600) #640,480
                                y_enemy= random.randint(50,400)
                                score = score+1
                                font= cv2.FONT_HERSHEY_SIMPLEX
                                color=(255,0,255)
                                text=cv2.putText(frame, "Score",(100,100),font,1,color,4,cv2,LINE_AA)
                                enemy()
                        except:
                            pass
        cv2.imshow('Hand Tracking', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            print(score)
            break
            
video.release()
cv2.destroyAllWindows()
                    

Found
Found
Found
13
